# Glossary

In [ ]:
# Set Environment Variable
%env PURVIEW_NAME=purview-sandbox

In [ ]:
# Commands available
!pv glossary --help

## Glossary

In [ ]:
# Create a Glossary
import json
cli_output = !pv glossary create --glossaryName='My First Glossary'
glossary = json.loads(''.join(cli_output))
glossary

In [ ]:
# Read a Glossary
!pv glossary read --glossaryGuid={glossary['guid']}

In [ ]:
# Delete a Glossary
!pv glossary delete --glossaryGuid={glossary['guid']}

In [ ]:
# Detailed Glossary
cli_output = !pv glossary read
glossary = json.loads(''.join(cli_output))
default_glossary = glossary[0]
!pv glossary readDetailed --glossaryGuid={default_glossary['guid']}

## Terms

In [ ]:
# Purge Glossary Terms
# terms = !pv glossary readTerms --glossaryGuid {default_glossary['guid']} 
# terms = json.loads(''.join(terms))

# for term in terms:
#     termGuid = term['guid']
#     if 'assignedEntities' in term:
#         for entity in term['assignedEntities']:
#             guid = entity['guid']
#             relationshipGuid = entity['relationshipGuid']
#             !pv glossary deleteAssignedEntities --termGuid {termGuid} --guid {guid} --relationshipGuid {relationshipGuid}
#     !pv glossary deleteTerm --termGuid {termGuid}

In [ ]:
# Create a Term
term = !pv glossary createTerm --glossaryGuid {default_glossary['guid']} --termName='My Term'
term = json.loads(''.join(term))
term

In [ ]:
# Read a Term
!pv glossary readTerm --termGuid={term['guid']}

In [ ]:
# Update a Term
# Need to fix
!pv glossary updateTerm --termGuid={term['guid']} --termName='New Name'

In [ ]:
# Delete a Term
!pv glossary deleteTerm --termGuid={term['guid']}

In [ ]:
# Create Terms (bulk)
!pv glossary createTerms --glossaryGuid {default_glossary['guid']} --termName='Term1' --termName='Term2' --termName='Term3'

In [ ]:
# Read Terms (bulk)
!pv glossary readTerms --glossaryGuid {default_glossary['guid']} 

In [ ]:
# Read Terms Headers (bulk)
!pv glossary readTermsHeaders --glossaryGuid {default_glossary['guid']}

In [ ]:
# Create Term Assignment
terms = !pv glossary readTerms --glossaryGuid {default_glossary['guid']} 
terms = json.loads(''.join(terms))
term = terms[0]
termGuid = term['guid']
guid = '85bc9357-f227-4ad9-9379-b0173766f241'
!pv glossary createAssignedEntities --termGuid={termGuid} --guid={guid}

In [ ]:
# Read Term Assignments
assignments = !pv glossary readAssignedEntities --termGuid={termGuid}
assignments = json.loads(''.join(assignments))
assignments

In [ ]:
# Delete Term Assignment
guid = assignments[0]['guid']
relationshipGuid = assignments[0]['relationshipGuid']
!pv glossary deleteAssignedEntities --termGuid={termGuid} --guid={guid} --relationshipGuid={relationshipGuid}    

In [ ]:
# Read Related Terms 
# (seeAlso = Related Terms; synonyms = Synonyms)
!pv glossary readTermsRelated --termGuid={termGuid}

## Category

In [ ]:
# Create a Category
category = !pv glossary createCategory --glossaryGuid {default_glossary['guid']}  --categoryName='Category Name'
category = json.loads(''.join(category))
category

In [ ]:
# Read a Category
!pv glossary readCategory --categoryGuid={category['guid']}

In [ ]:
# Delete a Category
!pv glossary deleteCategory --categoryGuid={category['guid']}

In [ ]:
# Read Related Categories
!pv glossary readCategoryRelated --categoryGuid={category['guid']}

In [ ]:
# Read Category Terms
!pv glossary readCategoryTerms --categoryGuid={category['guid']}

In [ ]:
# Read Categories (bulk)
!pv glossary readCategories --glossaryGuid={default_glossary['guid']}

In [ ]:
# Read Categories Headers (bulk)
!pv glossary readCategoriesHeaders --glossaryGuid={default_glossary['guid']}

In [ ]:
# Create Categories (bulk)
!pv glossary createCategories --glossaryGuid={default_glossary['guid']} --categoryName='Category1' --categoryName='Category2' --categoryName='Category3'

In [ ]:
#  Purge Categories
categories = !pv glossary readCategoriesHeaders --glossaryGuid={default_glossary['guid']}
categories = json.loads(''.join(categories))
categories
for category in categories:
    !pv glossary deleteCategory --categoryGuid={category['categoryGuid']}

In [ ]:
!pv glossary readTemplate